In [28]:
import torch

class TsetlinBase:
    def conjunctin_mul(self, X, W):
        self.twod_X = X.unsqueeze(0)
        self.X_neg = 1 - self.twod_X
        self.full_X = torch.cat((self.twod_X, self.X_neg), dim=1)
        self.matrix_X = self.full_X.repeat(W.shape[0], 1)
        self.mask = W > 0
        self.masked_X = torch.where(self.mask, self.matrix_X, torch.tensor(1))
        return torch.prod(self.masked_X, dim=1, keepdim=True).view(1,-1)

class TsetlinLayer(TsetlinBase):
    
    def __init__(self, in_dim, out_dim):
        w_pos = torch.randint(0, 2, (out_dim, in_dim,))
        w_neg = 1 - w_pos
        self.w = torch.cat((w_pos, w_neg), dim=1)
        self.out = None

    def forward(self, X):
        out = self.conjunctin_mul(X, self.w)
        self.out = out.squeeze(0)
        return self.out
    

    def backward(self, y):

        for i, expected_output in enumerate(y):
            if expected_output == self.out[i]:
                continue
            else:
                
                
                offending_row = self.w[i]




class TsetlinMachine:

    def __init__(self, in_dim):
        self.l1 = TsetlinLayer(in_dim, 10)
        self.l2 = TsetlinLayer(10, 1)
        self.out = None

    def forward(self, X):
        X = self.l1.forward(X)
        X = self.l2.forward(X)
        self.out = X.squeeze(0)
        return self.out
    

    def backward(self, y):
        if y == self.out:
            return
        





    

In [30]:

tm = TsetlinMachine(2)
tm.forward(torch.tensor([0,1]))

tensor([0])


tensor(0)

In [18]:
class TsetlinBase:
    def conjunctin_mul(self, X, W):
        self.twod_X = X.unsqueeze(0)
        print(self.twod_X)
        self.X_neg = 1 - self.twod_X
        print(self.X_neg)
        self.full_X = torch.cat((self.twod_X, self.X_neg), dim=1)
        print(self.full_X)
        self.matrix_X = self.full_X.repeat(W.shape[0], 1)
        self.mask = W > 0
        self.masked_X = torch.where(self.mask, self.matrix_X, torch.tensor(1))
        return torch.prod(self.masked_X, dim=1, keepdim=True).view(1,-1)
    
A = torch.tensor([1, 0])

B = torch.tensor([[2, -2, 3, 1],
                  [1, 2, -3, 4],
                  [-1, 0, 1, 2]])

TsetlinBase().conjunctin_mul(A, B).unsqueeze(0)

tensor([[1, 0]])
tensor([[0, 1]])
tensor([[1, 0, 0, 1]])


tensor([[[0, 0, 0]]])